In [20]:
import requests
import json
import IPython

url = 'https://iam.cloud.ibm.com/identity/token'
headers = {'Content-Type': 'application/x-www-form-urlencoded'}
data = {
    'grant_type': 'urn:ibm:params:oauth:grant-type:apikey',
    'apikey': api_key
}

response = requests.post(url, headers=headers, data=data)

if response.status_code != 200:
    print('Failed to get token:', response.status_code)
    print(response.text)
    exit()
else:
    print(response.json())

access_token = response.json()['access_token']

{'access_token': 'eyJraWQiOiIyMDI0MTAwMjA4NDIiLCJhbGciOiJSUzI1NiJ9.eyJpYW1faWQiOiJJQk1pZC02OTUwMDBKTFAwIiwiaWQiOiJJQk1pZC02OTUwMDBKTFAwIiwicmVhbG1pZCI6IklCTWlkIiwianRpIjoiMDAzZDZjYzQtNDhjNC00NzRhLThlNGEtNjk0MzA3OWFiMGVmIiwiaWRlbnRpZmllciI6IjY5NTAwMEpMUDAiLCJnaXZlbl9uYW1lIjoiQW1tYXIiLCJmYW1pbHlfbmFtZSI6IkFsbWFqZWQiLCJuYW1lIjoiQW1tYXIgQWxtYWplZCIsImVtYWlsIjoiYW1tYXJhbG1hamVkN0BnbWFpbC5jb20iLCJzdWIiOiJhbW1hcmFsbWFqZWQ3QGdtYWlsLmNvbSIsImF1dGhuIjp7InN1YiI6ImFtbWFyYWxtYWplZDdAZ21haWwuY29tIiwiaWFtX2lkIjoiSUJNaWQtNjk1MDAwSkxQMCIsIm5hbWUiOiJBbW1hciBBbG1hamVkIiwiZ2l2ZW5fbmFtZSI6IkFtbWFyIiwiZmFtaWx5X25hbWUiOiJBbG1hamVkIiwiZW1haWwiOiJhbW1hcmFsbWFqZWQ3QGdtYWlsLmNvbSJ9LCJhY2NvdW50Ijp7InZhbGlkIjp0cnVlLCJic3MiOiI1ODlmNTZhZDM2NTU0ZTkwYjAxNzJkYjE3M2VlNTg5NSIsImltc191c2VyX2lkIjoiMTI2Nzc5ODEiLCJmcm96ZW4iOnRydWUsImltcyI6IjI3NDg1NDAifSwiaWF0IjoxNzI5MjYwOTQ2LCJleHAiOjE3MjkyNjQ1NDYsImlzcyI6Imh0dHBzOi8vaWFtLmNsb3VkLmlibS5jb20vaWRlbnRpdHkiLCJncmFudF90eXBlIjoidXJuOmlibTpwYXJhbXM6b2F1dGg6Z3JhbnQtdHlwZTphcGlrZXkiL

In [21]:
from IPython.display import clear_output

!pip install ibm_watsonx_ai

clear_output()

In [22]:
import os
from ibm_watsonx_ai.foundation_models import Model
from ibm_watsonx_ai.foundation_models.utils.enums import DecodingMethods
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai import Credentials
# from dotenv import load_dotenv

Credentials = Credentials(
    url="https://eu-de.ml.cloud.ibm.com",
    token=access_token
)

paramaters = {
    "decoding_method": "greedy",
    "max_new_tokens": 100,
    "repetition_penalty": 1.0,
    "temperature": 1.0,
}

allam_model = Model(
    model_id="sdaia/allam-1-13b-instruct",
    credentials=Credentials,
    params=paramaters,
    project_id="8fa0b92f-a7b0-4dbc-bfa1-86e8ec65a58b"
)


In [23]:
def construct_prompt(query):
    System_prompt = "\n".join([
        "أنت مختص في اللغة العربية، ودورك هو تصحيح الأخطاء الإملائية والنحوية في الجمل العربية التي تتلقاها.",
        "يجب أن تقوم بالتغييرات الضرورية فقط، دون تغيير الكلمات أو المفردات إلا إذا كان ضروريًا للنحو.",
        "أجب فقط بالجملة المصححة دون إضافة أي شرح للتعديلات أو الأخطاء.",
        "مثال: إذا أعطيتك 'في نفس الوقت'، يجب أن ترد بـ'في الوقت نفسه' فقط، أو إذا أعطيتك 'لم أرى' ترد بـ'لم أرَ' فقط دون أي تعليق."
    ])
    prompt = f"{System_prompt}\n\nالجملة المراد تصحيحها: {query}\nالجملة الصحيحة:"
    return prompt


In [24]:
def get_watsonx_response(prompt):
    response = allam_model.generate_text(prompt)
    return response

In [26]:
!pip install --upgrade transformers
clear_output()

In [27]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, GenerationConfig
model_name = "Abdulmohsena/Faseeh"

tokenizer = AutoTokenizer.from_pretrained(model_name, src_lang="eng_Latn", tgt_lang="arb_Arab")
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
generation_config = GenerationConfig.from_pretrained(model_name)


In [ ]:
!pip install deep-translator
clear_output()

# Start testing

In [39]:
dummy = "I am searching for my future wife, it is not an easy task."

In [40]:
encoded_ar = tokenizer(dummy, return_tensors="pt")
generated_tokens = model.generate(**encoded_ar, generation_config=generation_config)

translated_sentence = tokenizer.decode(generated_tokens[0], skip_special_tokens=True)
print(translated_sentence)

أبحث عن امرأتي المستقبلة ليس من الأمر سهل


In [45]:
query = "ادخل الجملة المراد تصحيحها: " + translated_sentence
prompt = construct_prompt(query)
corrected_translated_sentence = get_watsonx_response(prompt)
print(response)

 أبحث عن امرأتي المستقبلية، ليس من الأمر سهل. 


In [46]:
from deep_translator import GoogleTranslator

# Translate from English to Arabic
translator = GoogleTranslator(source='en', target='ar')
translated_sentence_google = translator.translate(dummy)

In [47]:
print(f"Original: {dummy}")

print(f"Google Translation: {translated_sentence_google}")
print(f"Model Translation: {corrected_translated_sentence}")

Original: I am searching for my future wife, it is not an easy task.
Google Translation: أبحث عن زوجة المستقبل، وهي ليست مهمة سهلة.
Model Translation:  أبحث عن امرأتي المستقبلية، ليس من الأمر سهل. 
